In [3]:
import pandas as pd
import zipfile

In [1]:
! wget -P ../../data/raw https://aqua.kingcounty.gov/extranet/assessor/Parcel.zip
! wget -P ../../data/raw https://aqua.kingcounty.gov/extranet/assessor/Real%20Property%20Sales.zip
! wget -P ../../data/raw https://aqua.kingcounty.gov/extranet/assessor/Residential%20Building.zip

--2020-09-25 14:45:33--  https://aqua.kingcounty.gov/extranet/assessor/Parcel.zip
Resolving aqua.kingcounty.gov (aqua.kingcounty.gov)... 146.129.240.28
Connecting to aqua.kingcounty.gov (aqua.kingcounty.gov)|146.129.240.28|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 31113178 (30M) [application/x-zip-compressed]
Saving to: '../../data/raw/Parcel.zip.1'

     0K .......... .......... .......... .......... ..........  0%  827K 37s
    50K .......... .......... .......... .......... ..........  0%  940K 34s
   100K .......... .......... .......... .......... ..........  0% 1.70M 29s
   150K .......... .......... .......... .......... ..........  0% 1.68M 26s
   200K .......... .......... .......... .......... ..........  0% 5.57M 22s
   250K .......... .......... .......... .......... ..........  0% 2.23M 20s
   300K .......... .......... .......... .......... ..........  1% 1.57M 20s
   350K .......... .......... .......... .......... ..........  1% 1.62M 20s

--2020-09-25 14:45:47--  https://aqua.kingcounty.gov/extranet/assessor/Real%20Property%20Sales.zip
Resolving aqua.kingcounty.gov (aqua.kingcounty.gov)... 146.129.240.28
Connecting to aqua.kingcounty.gov (aqua.kingcounty.gov)|146.129.240.28|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 129493383 (123M) [application/x-zip-compressed]
Saving to: '../../data/raw/Real Property Sales.zip'

     0K .......... .......... .......... .......... ..........  0% 1.79M 69s
    50K .......... .......... .......... .......... ..........  0% 1.58M 73s
   100K .......... .......... .......... .......... ..........  0% 1.47M 77s
   150K .......... .......... .......... .......... ..........  0% 1.57M 77s
   200K .......... .......... .......... .......... ..........  0% 1.68M 76s
   250K .......... .......... .......... .......... ..........  0% 1.60M 77s
   300K .......... .......... .......... .......... ..........  0% 1.81M 75s
   350K .......... .......... .......... .....

 99600K .......... .......... .......... .......... .......... 78% 1.56M 13s
 99650K .......... .......... .......... .......... .......... 78% 1.86M 13s
 99700K .......... .......... .......... .......... .......... 78% 1.89M 13s
 99750K .......... .......... .......... .......... .......... 78% 8.42M 13s
 99800K .......... .......... .......... .......... .......... 78% 1.69M 13s
 99850K .......... .......... .......... .......... .......... 78% 1.86M 12s
 99900K .......... .......... .......... .......... .......... 79% 1.84M 12s
 99950K .......... .......... .......... .......... .......... 79% 1.79M 12s
100000K .......... .......... .......... .......... .......... 79% 1.75M 12s
100050K .......... .......... .......... .......... .......... 79% 7.62M 12s
100100K .......... .......... .......... .......... .......... 79% 1.82M 12s
100150K .......... .......... .......... .......... .......... 79% 2.12M 12s
100200K .......... .......... .......... .......... .......... 79% 1.90M 12s

--2020-09-25 14:46:47--  https://aqua.kingcounty.gov/extranet/assessor/Residential%20Building.zip
Resolving aqua.kingcounty.gov (aqua.kingcounty.gov)... 146.129.240.28
Connecting to aqua.kingcounty.gov (aqua.kingcounty.gov)|146.129.240.28|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24776122 (24M) [application/x-zip-compressed]
Saving to: '../../data/raw/Residential Building.zip'

     0K .......... .......... .......... .......... ..........  0% 1.51M 16s
    50K .......... .......... .......... .......... ..........  0% 1.49M 16s
   100K .......... .......... .......... .......... ..........  0% 1.53M 16s
   150K .......... .......... .......... .......... ..........  0% 2.06M 14s
   200K .......... .......... .......... .......... ..........  1% 1.95M 14s
   250K .......... .......... .......... .......... ..........  1% 1.79M 14s
   300K .......... .......... .......... .......... ..........  1% 2.50M 13s
   350K .......... .......... .......... .......

  6350K .......... .......... .......... .......... .......... 26% 29.4M 9s
  6400K .......... .......... .......... .......... .......... 26% 1.77M 9s
  6450K .......... .......... .......... .......... .......... 26% 1.78M 9s
  6500K .......... .......... .......... .......... .......... 27% 1.63M 8s
  6550K .......... .......... .......... .......... .......... 27% 1.85M 8s
  6600K .......... .......... .......... .......... .......... 27% 1.61M 8s
  6650K .......... .......... .......... .......... .......... 27% 24.0M 8s
  6700K .......... .......... .......... .......... .......... 27% 1.70M 8s
  6750K .......... .......... .......... .......... .......... 28% 1.76M 8s
  6800K .......... .......... .......... .......... .......... 28% 1.71M 8s
  6850K .......... .......... .......... .......... .......... 28% 1.75M 8s
  6900K .......... .......... .......... .......... .......... 28% 5.57M 8s
  6950K .......... .......... .......... .......... .......... 28% 2.21M 8s
  7000K ....

In [4]:
with zipfile.ZipFile('../../data/raw/Parcel.zip', 'r') as zip_ref:
    zip_ref.extractall('../../data/raw')
with zipfile.ZipFile('../../data/raw/Real Property Sales.zip', 'r') as zip_ref:
    zip_ref.extractall('../../data/raw')
with zipfile.ZipFile('../../data/raw/Residential Building.zip', 'r') as zip_ref:
    zip_ref.extractall('../../data/raw')

In [8]:
parcel = pd.read_csv("../../data/raw/EXTR_Parcel.csv", encoding='latin-1')
real_property_sales = pd.read_csv("../../data/raw/EXTR_RPSale.csv", encoding='latin-1', low_memory=False)
residential_building = pd.read_csv("../../data/raw/EXTR_ResBldg.csv", encoding='latin-1', low_memory=False)

In [10]:
residential_building.head()

,Major,Minor,BldgNbr,NbrLivingUnits,Address,BuildingNumber,Fraction,DirectionPrefix,StreetName,StreetType,...,FpMultiStory,FpFreestanding,FpAdditional,YrBuilt,YrRenovated,PcntComplete,Obsolescence,PcntNetCondition,Condition,AddnlCost
0,718680,55,3,1,5936 A 36TH AVE S 98118,5936,A,,36TH,AVE,...,0,0,0,2017,0,0,0,0,3,0
1,718680,55,2,1,5936 A 36TH AVE S 98118,5936,A,,36TH,AVE,...,0,0,0,2017,0,0,0,0,3,0
2,718680,55,4,1,5936 A 36TH AVE S 98118,5936,A,,36TH,AVE,...,0,0,0,2017,0,0,0,0,3,0
3,718680,55,1,1,5936 A 36TH AVE S 98118,5936,A,,36TH,AVE,...,0,0,0,2017,0,0,0,0,3,0
4,718680,55,5,1,5936 A 36TH AVE S 98118,5936,A,,36TH,AVE,...,0,0,0,2017,0,0,0,0,3,0
